In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [102]:
from comm_agents.data.reference_experiments import RefExperimentMass, RefExperimentCharge
from comm_agents.data.optimal_answers import get_alpha_star, get_phi_star
from ipywidgets import interact
import numpy as  np

In [97]:
# experiment parameters
GOLF_HOLE_LOC_M = 0.1
GOLF_HOLE_LOC_C = 1
PARAM_DICT = dict(
    m=[1e-20, 1e-20],
    q=[3e-12, -3e-12],
    m_ref=2e-12,
    v_ref=1,
    N=100,
    alpha=[0, 0],
    phi=[0, 0],
    dt=.01,
    d=.1,
    gravity=True)

In [114]:
alpha_range_re1 = np.linspace(0, 0.6, 20)
m_range_re1 = np.linspace(1e-20, 1e-12, 20)
alpha_range_re2 = np.linspace(0.5, 1, 20)
q_range_re2_1 = np.linspace(1e-16, 1e-14, 20)
q_range_re2_2 = np.linspace(-1e-16, -1e-14, 20)
N_reange = [100, 1000, 10000]

In [115]:
def get_re1_plot(alpha1, alpha2, m1, m2, N, alpha_star):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.angle = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    if alpha_star:
        rem.angle = get_alpha_star(rem)
    rem.set_initial_state()
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M)
    
interact(get_re1_plot, alpha1=alpha_range_re1, alpha2=alpha_range_re1,
         m1=m_range_re1, m2=m_range_re1, N=N_reange, alpha_star=False)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.031578947368421054, 0.06315789473684211, …

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2, N, alpha_star)>

In [117]:
def get_re2_plot(phi1=.5, phi2=.63, q1=4e-12, q2=-3e-12, N=100, phi_star=False):
    req = RefExperimentCharge(**PARAM_DICT)
    req.N = N
    req.angle = np.array([phi1*np.pi, phi2*np.pi])
    req.q = np.array([q1, q2])
    if phi_star:
        req.angle = get_phi_star(req)
    req.set_initial_state()
    req.run()
    req.visualize(golf_hole_loc=.1)
interact(get_re2_plot, phi1=alpha_range_re2, phi2=alpha_range_re2,
        q1=q_range_re2_1, q2=q_range_re2_2, N=N_reange, phi_star=False)

interactive(children=(Dropdown(description='phi1', options=(0.5, 0.5263157894736842, 0.5526315789473684, 0.578…

<function __main__.get_re2_plot(phi1=0.5, phi2=0.63, q1=4e-12, q2=-3e-12, N=100, phi_star=False)>

In [78]:
alpha_star = get_alpha_star(rem)
phi_star = get_phi_star(req)

2020-10-12 20:29:22.399 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.400 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.405 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.406 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.413 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.413 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.419 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0

2020-10-12 20:29:22.646 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.648 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.656 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.657 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.664 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0
2020-10-12 20:29:22.665 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 1 did not cross line with x=0
2020-10-12 20:29:22.672 | DEBUG    | comm_agents.data.optimal_answers:get_single_charge_loss_value:30 - Particle 0 did not cross line with x=0